> Note: Change executor as 1 for this lab

## Load data to CosmosDB

For Spark 3 Cosmos DB connector has slightly different configuration. Please select different snippest if a Spark 3.1 + pool is attached.

In [24]:
# Write a Spark DataFrame into a Cosmos DB container
# To select a preferred list of regions in a multi-region Cosmos DB account, add .option("spark.cosmos.preferredRegions", "<Region1>,<Region2>")
import datetime
import time
import uuid
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType, LongType


df = spark.read.load('abfss://dev-synapse@hyundevsynapsestorage.dfs.core.windows.net/curated/mtcsdevsqlsrv/wwi/saleslt/customer.parquet/part-00000-1a3142c7-8399-4a96-b244-57a43b3c4174-c000.snappy.parquet', format='parquet')
uuidUdf= udf(lambda : str(uuid.uuid4()),StringType())
df = df \
  .withColumn("id", uuidUdf()) 
  
# For Spark 3.1 +
df.write\
    .format("cosmos.oltp")\
    .option("spark.synapse.linkedService", "link_cos_wwi")\
    .option("spark.cosmos.container", "customer")\
    .option("spark.cosmos.write.strategy", "ItemOverwrite")\
    .option("spark.cosmos.write.bulk.enabled", "true")\
    .mode('append')\
    .save()

In [23]:
df.count()